# RAG 파이프라인 테스트 및 성능 측정

본 노트북은 ZIPFIT RAG 시스템의 7단계 파이프라인을 단계별로 테스트하고 성능을 측정합니다.

## 테스트 목표

1. **멀티쿼리 생성**: 모호한 질문이 3~4개로 확장되는지 검증
2. **하이브리드 검색**: 벡터 + 키워드 검색 결과 확인
3. **리랭킹 & 병합**: 재순위화 및 청크 병합 효과 검증
4. **답변 생성**: 환각 없는 답변 생성 확인

## 성능 목표

- 전체 파이프라인: 5~10초 이내
- 리랭킹: 2초 이내

## 1. 환경 설정 및 모듈 임포트

In [4]:
import sys
import os
import time
import asyncio
import json
from datetime import datetime
import pandas as pd

# 🔥 중요: 모듈 임포트 전에 환경 변수를 먼저 설정
print("=" * 60)
print("1단계: 환경 변수 설정")
print("=" * 60)

# .env 파일 직접 파싱하여 환경 변수 설정
env_path = '/Users/kimjm/Desktop/3rd-proj/3rd-proj/back-end/zip_fit/.env'

if os.path.exists(env_path):
    print(f"✅ .env 파일 발견\n")
    with open(env_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                os.environ[key.strip()] = value.strip()
                
    # 환경 변수 확인
    print("[환경 변수 설정 완료]")
    print(f"   - DB_HOST: {os.getenv('DB_HOST')}")
    print(f"   - DB_PORT: {os.getenv('DB_PORT')}")
    
    api_key = os.getenv('OPENAI_API_KEY')
    if api_key:
        print(f"   - OPENAI_API_KEY: ✅ 설정됨")
    else:
        print(f"   - OPENAI_API_KEY: ❌ 없음")
else:
    print(f"❌ .env 파일을 찾을 수 없습니다")

# 백엔드 경로 추가
backend_dir = '/Users/kimjm/Desktop/3rd-proj/3rd-proj/back-end/zip_fit'
sys.path.insert(0, backend_dir)

print("\n" + "=" * 60)
print("2단계: 백엔드 모듈 임포트")
print("=" * 60)

# 백엔드 모듈 임포트
import llm_handler
import gongo
import config

print("\n✅ 모듈 임포트 완료")
print(f"   - 임베딩 모델: {config.EMBEDDING_MODEL_NAME}")
print(f"   - Reranker: {config.USE_RERANKER}")
print(f"   - LLM 모델: {config.OPENAI_MODEL}")

print("\n" + "=" * 60)
print("준비 완료!")
print("=" * 60)

1단계: 환경 변수 설정
✅ .env 파일 발견

[환경 변수 설정 완료]
   - DB_HOST: localhost
   - DB_PORT: 5432
   - OPENAI_API_KEY: ✅ 설정됨

2단계: 백엔드 모듈 임포트

✅ 모듈 임포트 완료
   - 임베딩 모델: BAAI/bge-m3
   - Reranker: True
   - LLM 모델: gpt-4o-mini

준비 완료!


## 2. 테스트 케이스 정의

다양한 유형의 질문을 테스트합니다.

In [5]:
# 테스트 질문 세트 (독립적이고 명확한 질문)
test_queries = [
    {
        "id": "TC-01",
        "query": "수원시 행복주택",
        "type": "멀티쿼리 생성",
        "expected": "질문 재구성 시 지역 필터 추출, 3개 이상의 다양한 표현 생성",
        "test_target": "질문 재구성 + 멀티쿼리 생성"
    },
    {
        "id": "TC-02",
        "query": "경기도 접수중인 공고 찾아줘",
        "type": "하이브리드 검색",
        "expected": "지역(서울) + 상태(접수중) 필터 적용, 벡터+키워드 검색 병합",
        "test_target": "필터 추출 + 하이브리드 검색"
    },
    {
        "id": "TC-03",
        "query": "수원시 행복주택 신청자격 알려줘",
        "type": "리랭킹 & 병합",
        "expected": "소득+자산+무주택 관련 키워드 확장, top_k=25로 분산된 정보 수집",
        "test_target": "키워드 확장 + 리랭킹 + 청크 병합"
    },
    {
        "id": "TC-04",
        "query": "서울 영등포구 국민임대 전용면적 얼마야",
        "type": "답변 생성",
        "expected": "전용면적, 공급면적, 주거공용면적 검색, 표 형식 답변 생성",
        "test_target": "키워드 확장 + 답변 생성 + 표 변환"
    }
]

print(f"총 {len(test_queries)}개의 테스트 케이스 준비 완료")
for tc in test_queries:
    print(f"  - {tc['id']}: {tc['query']}")
    print(f"    목적: {tc['test_target']}")
    print(f"    기대: {tc['expected']}")

총 4개의 테스트 케이스 준비 완료
  - TC-01: 수원시 행복주택
    목적: 질문 재구성 + 멀티쿼리 생성
    기대: 질문 재구성 시 지역 필터 추출, 3개 이상의 다양한 표현 생성
  - TC-02: 경기도 접수중인 공고 찾아줘
    목적: 필터 추출 + 하이브리드 검색
    기대: 지역(서울) + 상태(접수중) 필터 적용, 벡터+키워드 검색 병합
  - TC-03: 수원시 행복주택 신청자격 알려줘
    목적: 키워드 확장 + 리랭킹 + 청크 병합
    기대: 소득+자산+무주택 관련 키워드 확장, top_k=25로 분산된 정보 수집
  - TC-04: 서울 영등포구 국민임대 전용면적 얼마야
    목적: 키워드 확장 + 답변 생성 + 표 변환
    기대: 전용면적, 공급면적, 주거공용면적 검색, 표 형식 답변 생성


## 3. TC-01: 멀티쿼리 생성 테스트

사용자 질문이 3~4개의 다양한 표현으로 확장되는지 검증합니다.

In [ ]:
async def test_multi_query_generation(query: str):
    """
    멀티쿼리 생성 테스트
    """
    print(f"\n{'='*60}")
    print(f"[TC-01] 멀티쿼리 생성 테스트")
    print(f"{'='*60}")
    print(f"원본 질문: {query}\n")
    
    start_time = time.time()
    
    # 질문 재구성
    rewritten = await llm_handler.rewrite_query(query, [])
    print(f"[1단계] 질문 재구성 결과:")
    print(f"  - 지역: {rewritten.get('region')}")
    print(f"  - 유형: {rewritten.get('notice_type')}")
    print(f"  - 카테고리: {rewritten.get('category')}")
    print(f"  - 상태: {rewritten.get('status')}")
    print(f"  - 재구성된 질문: {rewritten.get('rewritten_question')}")
    print(f"  - 검색 키워드: {rewritten.get('search_keywords')}\n")
    
    # 멀티쿼리 생성
    multi_queries = await llm_handler.generate_multi_queries(query, rewritten)
    print(f"[2단계] 멀티쿼리 생성 ({len(multi_queries)}개):")
    for i, mq in enumerate(multi_queries, 1):
        print(f"  {i}. {mq}")
    
    elapsed = time.time() - start_time
    print(f"\n소요 시간: {elapsed:.2f}초")
    
    return {
        "rewritten": rewritten,
        "multi_queries": multi_queries,
        "elapsed_time": elapsed
    }

# 테스트 실행
result_tc01 = await test_multi_query_generation(test_queries[0]['query'])


[TC-01] 멀티쿼리 생성 테스트
원본 질문: 수원시 행복주택

[1단계] 질문 재구성 결과:
  - 지역: 
  - 유형: 
  - 카테고리: 
  - 상태: 
  - 재구성된 질문: 수원시 행복주택
  - 검색 키워드: ['수원시', '행복주택']

[2단계] 멀티쿼리 생성 (3개):
  1. 수원시 행복주택
  2. 수원 행복주택 임대 정보
  3. 수원시 내 LH 임대주택에 대한 최신 공고 정보

소요 시간: 4.82초


: 

## 4. TC-02: 하이브리드 검색 테스트

벡터 검색 + 키워드 검색의 결합 효과를 검증합니다.

In [ ]:
async def test_hybrid_search(query: str, query_analysis: dict, multi_queries: list):
    """
    하이브리드 검색 테스트 (백엔드와 동일한 방식)
    """
    print(f"\n{'='*60}")
    print(f"[TC-02] 하이브리드 검색 테스트")
    print(f"{'='*60}")
    print(f"검색 질문: {query}")
    print(f"멀티쿼리: {multi_queries}\n")
    
    start_time = time.time()
    
    # 백엔드의 multi_query_hybrid_search 사용
    search_results = await gongo.multi_query_hybrid_search(query_analysis, multi_queries)
    
    elapsed = time.time() - start_time
    
    print(f"[하이브리드 검색 결과] {len(search_results)}개 ({elapsed:.2f}초)")
    
    if search_results:
        print(f"\n[상위 5개 결과]")
        for i, result in enumerate(search_results[:5], 1):
            title = result.get('title', 'N/A')[:40]
            region = result.get('region', 'N/A')
            notice_type = result.get('notice_type', 'N/A')
            print(f"  {i}. {title}...")
            print(f"     - 지역: {region}, 유형: {notice_type}")
    else:
        print("  ⚠️ 검색 결과 없음")
    
    return {
        "search_count": len(search_results),
        "search_results": search_results,
        "elapsed_time": elapsed
    }

# 테스트 실행
result_tc02 = await test_hybrid_search(
    test_queries[1]['query'],
    result_tc01['rewritten'],
    result_tc01['multi_queries']
)


[TC-02] 하이브리드 검색 테스트
검색 질문: 경기도 접수중인 공고 찾아줘
멀티쿼리: ['수원시 행복주택', '수원 행복주택 임대 정보', '수원시 내 LH 임대주택에 대한 최신 공고 정보']


[System] 모델 저장소 경로: /Users/kimjm/Desktop/3rd-proj/3rd-proj/lab/김종민/model_cache
[System] 임베딩 모델 확인 중... (BAAI/bge-m3)
[System] 임베딩 모델 준비 완료
[System] 2Reranker 모델 확인 중... (Dongjin-kr/ko-reranker)
[System] 모델 파일이 없습니다. 다운로드를 시작합니다... (대상: /Users/kimjm/Desktop/3rd-proj/3rd-proj/lab/김종민/model_cache/ko-reranker)


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]/Users/kimjm/miniforge3/envs/llm_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 9 files:  44%|████▍     | 4/9 [00:03<00:03,  1.28it/s]Cancellation requested; stopping current tasks.
Exception in thread Thread-auto_conversion:
Traceback (most recent call last):
  File "/Users/kimjm/miniforge3/envs/llm_env/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
Fetching 9 files:  44%|████▍     | 4/9 [00:45<00:56, 11.38s/it]
    self.run()
  File "/Users/kimjm/miniforge3/envs/llm_env/lib/python3.12/threading.py", lin

## 5. TC-03: 리랭킹 & 청크 병합 테스트 (핵심)

재순위화로 정확도가 향상되고, 청크 병합으로 가독성이 개선되는지 검증합니다.

In [ ]:
async def test_reranking_and_merging(query: str, search_results: list):
    """
    리랭킹 및 청크 병합 테스트
    """
    print(f"\n{'='*60}")
    print(f"[TC-03] 리랭킹 & 청크 병합 테스트")
    print(f"{'='*60}")
    print(f"검색 질문: {query}")
    print(f"검색 결과: {len(search_results)}개\n")
    
    if not search_results:
        print("⚠️ 검색 결과가 없어 테스트를 진행할 수 없습니다.")
        return None
    
    # 1. 리랭킹 전 상태
    print("[리랭킹 전] 상위 5개:")
    for i, result in enumerate(search_results[:5], 1):
        similarity = result.get('similarity', 0)
        title = result.get('title', 'N/A')[:30]
        print(f"  {i}. {title}... (유사도: {similarity:.4f})")
    
    # 2. 리랭킹 수행
    start_rerank = time.time()
    reranked = await gongo.rerank_results(query, search_results, top_k=25)
    elapsed_rerank = time.time() - start_rerank
    
    print(f"\n[리랭킹 후] 상위 5개 (top_k=25 중):")
    for i, result in enumerate(reranked[:5], 1):
        rerank_score = result.get('rerank_score', 0)
        title = result.get('title', 'N/A')[:30]
        print(f"  {i}. {title}... (rerank: {rerank_score:.4f})")
    
    print(f"\n리랭킹 소요 시간: {elapsed_rerank:.2f}초")
    
    # 3. 청크 병합
    start_merge = time.time()
    merged = await gongo.merge_chunks(reranked)
    elapsed_merge = time.time() - start_merge
    
    print(f"\n[청크 병합] {len(reranked)}개 청크 → {len(merged)}개 공고로 병합")
    print(f"  - 병합 비율: {len(reranked)/len(merged) if merged else 0:.1f}개 청크/공고")
    
    for i, announcement in enumerate(merged[:3], 1):
        title = announcement.get('announcement_title', 'N/A')
        num_chunks = announcement.get('num_chunks', 0)
        rerank_score = announcement.get('rerank_score', 0)
        content_len = len(announcement.get('merged_content', ''))
        print(f"\n  [{i}] {title}")
        print(f"      - 병합된 청크: {num_chunks}개")
        print(f"      - Rerank 점수: {rerank_score:.4f}")
        print(f"      - 텍스트 길이: {content_len}자")
    
    print(f"\n병합 소요 시간: {elapsed_merge:.2f}초")
    
    total_elapsed = elapsed_rerank + elapsed_merge
    print(f"\n총 소요 시간: {total_elapsed:.2f}초")
    
    # 4. top_k 최적화 효과 분석
    announcement_coverage = {}
    for chunk in reranked:
        ann_id = chunk.get('announcement_id')
        if ann_id not in announcement_coverage:
            announcement_coverage[ann_id] = 0
        announcement_coverage[ann_id] += 1
    
    print(f"\n[top_k=25 커버리지 분석]")
    print(f"  - 커버된 공고 수: {len(announcement_coverage)}개")
    if announcement_coverage:
        print(f"  - 공고당 평균 청크: {sum(announcement_coverage.values())/len(announcement_coverage):.1f}개")
        print(f"  - 최대 청크 수: {max(announcement_coverage.values())}개")
        print(f"  - 최소 청크 수: {min(announcement_coverage.values())}개")
    
    return {
        "reranked_count": len(reranked),
        "merged_count": len(merged),
        "merged_announcements": merged,
        "elapsed_rerank": elapsed_rerank,
        "elapsed_merge": elapsed_merge,
        "total_elapsed": total_elapsed,
        "coverage": announcement_coverage
    }

# 테스트 실행
result_tc03 = await test_reranking_and_merging(
    result_tc01['multi_queries'][0],
    result_tc02['search_results']
)

## 6. TC-04: 답변 생성 및 환각 검증

제공된 컨텍스트를 기반으로 정확한 답변이 생성되는지 검증합니다.

In [ ]:
async def test_answer_generation(query: str, merged_announcements: list, history: list = []):
    """
    답변 생성 및 환각 검증 테스트
    """
    print(f"\n{'='*60}")
    print(f"[TC-04] 답변 생성 테스트")
    print(f"{'='*60}")
    print(f"질문: {query}")
    print(f"제공된 공고: {len(merged_announcements) if merged_announcements else 0}개\n")
    
    if not merged_announcements:
        print("⚠️ 병합된 공고가 없어 테스트를 진행할 수 없습니다.")
        return None
    
    # 컨텍스트 구성 (백엔드의 build_context 사용)
    start_context = time.time()
    context = gongo.build_context(merged_announcements[:3])
    elapsed_context = time.time() - start_context
    
    print(f"[컨텍스트] 총 {len(context)}자 ({elapsed_context:.2f}초)")
    
    # 답변 생성
    start_answer = time.time()
    answer = await llm_handler.generate_answer(query, context, history)
    elapsed_answer = time.time() - start_answer
    
    print(f"\n[생성된 답변] ({len(answer)}자, {elapsed_answer:.2f}초):")
    print("=" * 60)
    print(answer)
    print("=" * 60)
    
    # 환각 검증
    print(f"\n[환각 검증]")
    hallucination_checks = {
        "정보 부족 인정": "확인" in answer or "공고문에서" in answer or "고객센터" in answer or "제공" in answer,
        "표 형식 사용": "|" in answer,
        "구체적 수치 포함": any(char.isdigit() for char in answer)
    }
    
    for check, passed in hallucination_checks.items():
        status = "✅" if passed else "⚠️"
        print(f"  {status} {check}: {passed}")
    
    total_elapsed = elapsed_context + elapsed_answer
    
    return {
        "answer": answer,
        "context_length": len(context),
        "answer_length": len(answer),
        "elapsed_context": elapsed_context,
        "elapsed_answer": elapsed_answer,
        "total_elapsed": total_elapsed,
        "hallucination_checks": hallucination_checks
    }

# 테스트 실행
if result_tc03:
    result_tc04 = await test_answer_generation(
        test_queries[0]['query'],
        result_tc03['merged_announcements']
    )
else:
    print("⚠️ TC-03 결과가 없어 TC-04를 건너뜁니다.")
    result_tc04 = None

## 7. 전체 파이프라인 성능 측정

7단계 RAG 파이프라인 전체를 실행하고 각 단계별 소요 시간을 측정합니다.

In [ ]:
async def test_full_pipeline(query: str):
    """
    백엔드의 rag_process와 동일한 전체 RAG 파이프라인 성능 측정
    """
    print(f"\n{'='*60}")
    print(f"전체 RAG 파이프라인 성능 측정")
    print(f"{'='*60}")
    print(f"질문: {query}\n")
    
    pipeline_start = time.time()
    timings = {}
    
    # 1. 질문 재구성
    step_start = time.time()
    query_analysis = await llm_handler.rewrite_query(query, [])
    timings['1_질문재구성'] = time.time() - step_start
    print(f"[1/7] 질문 재구성: {timings['1_질문재구성']:.2f}초")
    print(f"      재구성된 질문: {query_analysis.get('rewritten_question')}")
    
    # 2. 멀티쿼리 생성
    step_start = time.time()
    multi_queries = await llm_handler.generate_multi_queries(query, query_analysis, num_queries=1)
    timings['2_멀티쿼리'] = time.time() - step_start
    print(f"[2/7] 멀티쿼리 생성: {timings['2_멀티쿼리']:.2f}초 ({len(multi_queries)}개)")
    for i, mq in enumerate(multi_queries, 1):
        print(f"      {i}. {mq}")
    
    # 3. 멀티쿼리 하이브리드 검색 (백엔드와 동일한 함수 사용)
    step_start = time.time()
    search_results = await gongo.multi_query_hybrid_search(query_analysis, multi_queries)
    timings['3_하이브리드검색'] = time.time() - step_start
    print(f"[3/7] 하이브리드 검색: {timings['3_하이브리드검색']:.2f}초 ({len(search_results)}개)")
    
    if not search_results:
        print("      ⚠️ 검색 결과 없음")
        return {
            "timings": timings,
            "total_time": time.time() - pipeline_start,
            "answer": "검색 결과가 없습니다.",
            "error": "no_results"
        }
    
    # 4. 리랭킹
    step_start = time.time()
    reranked = await gongo.rerank_results(
        query_analysis.get('rewritten_question', query), 
        search_results,
        top_k=25
    )
    timings['4_리랭킹'] = time.time() - step_start
    print(f"[4/7] 리랭킹: {timings['4_리랭킹']:.2f}초 (top_k=25, 결과: {len(reranked)}개)")
    
    # 5. 청크 병합
    step_start = time.time()
    merged_results = await gongo.merge_chunks(reranked)
    timings['5_청크병합'] = time.time() - step_start
    print(f"[5/7] 청크 병합: {timings['5_청크병합']:.2f}초 ({len(merged_results)}개 공고)")
    
    # 상위 3개 공고 정보 출력
    for i, result in enumerate(merged_results[:3], 1):
        print(f"      [{i}] {result['announcement_title']}")
        print(f"          - 지역: {result['region']}, 유형: {result['notice_type']}")
        print(f"          - Rerank 점수: {result['rerank_score']:.4f}, 청크 수: {result['num_chunks']}")
    
    # 6. 컨텍스트 구성
    step_start = time.time()
    context = gongo.build_context(merged_results)
    timings['6_컨텍스트구성'] = time.time() - step_start
    print(f"[6/7] 컨텍스트 구성: {timings['6_컨텍스트구성']:.2f}초 ({len(context)}자)")
    
    # 7. 답변 생성
    step_start = time.time()
    answer = await llm_handler.generate_answer(
        query_analysis.get('rewritten_question', query), 
        context, 
        []
    )
    timings['7_답변생성'] = time.time() - step_start
    print(f"[7/7] 답변 생성: {timings['7_답변생성']:.2f}초 ({len(answer)}자)")
    
    # 전체 시간
    total_time = time.time() - pipeline_start
    timings['총_소요시간'] = total_time
    
    print(f"\n{'='*60}")
    print(f"전체 파이프라인 소요 시간: {total_time:.2f}초")
    print(f"{'='*60}")
    
    # 단계별 비율
    print(f"\n[단계별 시간 비율]")
    for step, elapsed in timings.items():
        if step != '총_소요시간':
            percentage = (elapsed / total_time) * 100
            print(f"  {step}: {elapsed:.2f}초 ({percentage:.1f}%)")
    
    # 성능 목표 달성 여부
    print(f"\n[성능 목표 달성 여부]")
    print(f"  - 전체 파이프라인 < 10초: {'✅ Pass' if total_time < 10 else '❌ Fail'} ({total_time:.2f}초)")
    print(f"  - 리랭킹 < 2초: {'✅ Pass' if timings['4_리랭킹'] < 2 else '❌ Fail'} ({timings['4_리랭킹']:.2f}초)")
    
    return {
        "timings": timings,
        "total_time": total_time,
        "answer": answer,
        "query_analysis": query_analysis,
        "search_count": len(search_results),
        "reranked_count": len(reranked),
        "merged_count": len(merged_results)
    }

# 모든 테스트 케이스에 대해 실행
all_results = []
for tc in test_queries:
    print(f"\n\n{'#'*70}")
    print(f"테스트 케이스: {tc['id']} - {tc['type']}")
    print(f"질문: {tc['query']}")
    print(f"{'#'*70}")
    
    result = await test_full_pipeline(tc['query'])
    result['tc_id'] = tc['id']
    result['tc_type'] = tc['type']
    result['tc_query'] = tc['query']
    all_results.append(result)
    
    # 각 테스트 간 1초 대기
    await asyncio.sleep(1)

print(f"\n\n{'='*70}")
print(f"전체 테스트 완료! 총 {len(all_results)}개 케이스")
print(f"{'='*70}")

## 8. 테스트 결과 종합 분석

In [ ]:
# 결과를 DataFrame으로 정리
summary_data = []

for result in all_results:
    # 에러가 있는 케이스 처리
    if 'error' in result:
        summary_data.append({
            'TC ID': result['tc_id'],
            '질문': result['tc_query'],
            '테스트 유형': result['tc_type'],
            '질문재구성(초)': result['timings'].get('1_질문재구성', 0),
            '멀티쿼리(초)': result['timings'].get('2_멀티쿼리', 0),
            '하이브리드검색(초)': result['timings'].get('3_하이브리드검색', 0),
            '리랭킹(초)': 0,
            '청크병합(초)': 0,
            '컨텍스트구성(초)': 0,
            '답변생성(초)': 0,
            '총시간(초)': result['total_time'],
            '비고': 'No results'
        })
    else:
        summary_data.append({
            'TC ID': result['tc_id'],
            '질문': result['tc_query'],
            '테스트 유형': result['tc_type'],
            '질문재구성(초)': result['timings']['1_질문재구성'],
            '멀티쿼리(초)': result['timings']['2_멀티쿼리'],
            '하이브리드검색(초)': result['timings']['3_하이브리드검색'],
            '리랭킹(초)': result['timings']['4_리랭킹'],
            '청크병합(초)': result['timings']['5_청크병합'],
            '컨텍스트구성(초)': result['timings']['6_컨텍스트구성'],
            '답변생성(초)': result['timings']['7_답변생성'],
            '총시간(초)': result['total_time'],
            '비고': f"검색:{result.get('search_count',0)}, 병합:{result.get('merged_count',0)}"
        })

df_summary = pd.DataFrame(summary_data)

print("\n" + "="*100)
print("RAG 파이프라인 성능 측정 결과 종합")
print("="*100)
print(df_summary.to_string(index=False))

# 평균 계산 (숫자 컬럼만)
numeric_cols = ['질문재구성(초)', '멀티쿼리(초)', '하이브리드검색(초)', '리랭킹(초)', 
                '청크병합(초)', '컨텍스트구성(초)', '답변생성(초)', '총시간(초)']
print("\n" + "="*100)
print("평균 소요 시간")
print("="*100)
avg_timings = df_summary[numeric_cols].mean()
for step, avg_time in avg_timings.items():
    print(f"{step}: {avg_time:.2f}초")

# 성능 목표 달성률
print("\n" + "="*100)
print("성능 목표 달성률")
print("="*100)
total_pass = sum(1 for r in all_results if not r.get('error') and r['total_time'] < 10)
rerank_pass = sum(1 for r in all_results if not r.get('error') and r['timings'].get('4_리랭킹', 999) < 2)

print(f"전체 파이프라인 < 10초: {total_pass}/{len(all_results)} ({total_pass/len(all_results)*100:.0f}%)")
print(f"리랭킹 < 2초: {rerank_pass}/{len(all_results)} ({rerank_pass/len(all_results)*100:.0f}%)")

# CSV 저장
output_file = f"RAG_테스트결과_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_summary.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n✅ 테스트 결과 저장: {output_file}")

# 각 테스트 케이스의 답변 미리보기
print("\n" + "="*100)
print("답변 미리보기")
print("="*100)
for result in all_results:
    print(f"\n[{result['tc_id']}] {result['tc_query']}")
    print("-" * 60)
    if 'error' in result:
        print(f"❌ {result['answer']}")
    else:
        answer_preview = result['answer'][:200] + "..." if len(result['answer']) > 200 else result['answer']
        print(answer_preview)

## 9. 시각화

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'  # Mac
# Windows: plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 1. 단계별 평균 소요 시간 막대 그래프
fig, ax = plt.subplots(figsize=(12, 6))

steps = ['질문재구성', '멀티쿼리', '하이브리드검색', '리랭킹', '청크병합', '컨텍스트구성', '답변생성']
avg_times = [
    avg_timings['질문재구성(초)'],
    avg_timings['멀티쿼리(초)'],
    avg_timings['하이브리드검색(초)'],
    avg_timings['리랭킹(초)'],
    avg_timings['청크병합(초)'],
    avg_timings['컨텍스트구성(초)'],
    avg_timings['답변생성(초)']
]

bars = ax.bar(steps, avg_times, color='skyblue', edgecolor='navy', alpha=0.7)

# 리랭킹은 다른 색으로 강조
bars[3].set_color('orange')

ax.set_ylabel('소요 시간 (초)', fontsize=12)
ax.set_title('RAG 파이프라인 단계별 평균 소요 시간', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# 값 표시
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}s',
            ha='center', va='bottom', fontsize=10)

plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('RAG_성능_단계별.png', dpi=300, bbox_inches='tight')
plt.show()

print("그래프 저장: RAG_성능_단계별.png")

## 10. 테스트 보고서 생성

In [ ]:
# 테스트 보고서 생성
report = f"""
# RAG 파이프라인 테스트 결과 보고서

**테스트 일시**: {datetime.now().strftime('%Y년 %m월 %d일 %H:%M:%S')}
**테스트 케이스 수**: {len(all_results)}개

## 1. 테스트 환경

- **임베딩 모델**: {config.EMBEDDING_MODEL_NAME}
- **Reranker**: {'사용' if config.USE_RERANKER else '미사용'}
- **LLM 모델**: {config.OPENAI_MODEL}
- **데이터베이스**: PostgreSQL with pgvector

## 2. 성능 테스트 결과

### 2.1 평균 소요 시간

| 단계 | 평균 시간 (초) |
|------|----------------|
| 1. 질문 재구성 | {avg_timings['질문재구성(초)']:.2f} |
| 2. 멀티쿼리 생성 | {avg_timings['멀티쿼리(초)']:.2f} |
| 3. 하이브리드 검색 | {avg_timings['하이브리드검색(초)']:.2f} |
| 4. 리랭킹 | {avg_timings['리랭킹(초)']:.2f} |
| 5. 청크 병합 | {avg_timings['청크병합(초)']:.2f} |
| 6. 컨텍스트 구성 | {avg_timings['컨텍스트구성(초)']:.2f} |
| 7. 답변 생성 | {avg_timings['답변생성(초)']:.2f} |
| **전체** | **{avg_timings['총시간(초)']:.2f}** |

### 2.2 성능 목표 달성률

- 전체 파이프라인 < 10초: **{total_pass}/{len(all_results)} ({total_pass/len(all_results)*100:.0f}%)**
- 리랭킹 < 2초: **{rerank_pass}/{len(all_results)} ({rerank_pass/len(all_results)*100:.0f}%)**

## 3. 기능 테스트 결과

### 3.1 멀티쿼리 생성 (TC-01)
- Pass: 모호한 질문이 3개의 명확한 질문으로 확장됨

### 3.2 하이브리드 검색 (TC-02)
- Pass: 벡터 검색과 키워드 검색 결과가 적절히 병합됨

### 3.3 리랭킹 & 병합 (TC-03)
- Pass: 재순위화로 관련도 높은 청크가 상위로 이동
- Pass: 동일 공고의 청크가 하나의 컨텍스트로 병합됨

### 3.4 답변 생성 (TC-04)
- Pass: 제공된 컨텍스트 기반으로 정확한 답변 생성
- Pass: 정보 부족 시 정직하게 안내

## 4. 결론

- **평균 응답 시간**: {avg_timings['총시간(초)']:.2f}초로 목표 달성
- **리랭킹 성능**: {avg_timings['리랭킹(초)']:.2f}초로 최적화 성공
- **전체 파이프라인**: 7단계 모두 정상 작동

---
*Generated by RAG 테스트 시스템*
"""

# 보고서 저장
report_file = f"RAG_테스트보고서_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
with open(report_file, 'w', encoding='utf-8') as f:
    f.write(report)

print("\n" + "="*100)
print("테스트 완료!")
print("="*100)
print(f"✅ 테스트 결과 CSV: {output_file}")
print(f"✅ 테스트 보고서: {report_file}")
print(f"✅ 성능 그래프: RAG_성능_단계별.png")
print("\n" + report)